In [4]:
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
from susy_qm import calculate_Hamiltonian

In [115]:
cutoff = 8
potential = 'DW'
shots = 10240

In [116]:
# Calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])[:4]

num_qubits = int(1 + np.log2(cutoff))

In [138]:
dev = qml.device("default.qubit", wires=num_qubits, shots=1024, seed=42)
@qml.qnode(dev)
def circuit(params):
    
    qml.RY(params[0], wires=[num_qubits-3])
    qml.RY(params[1], wires=[num_qubits-1])
    qml.CRY(params[2], wires=[num_qubits-1, num_qubits-2])
    qml.RY(params[3], wires=[num_qubits-2])
    #qml.RY(params[4], wires=[num_qubits-1])
        
    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [139]:
params =  [
            6.0766583400547685,
            1.2478077864113921,
            5.586453867584675,
            0.33541860443761573
        ]

In [140]:
lowest = 1231321.0
for i in range(1024):
    e = circuit(params)
    if e < lowest:
        lowest = e
        print(i, lowest)

0 0.9502708778895633
1 0.9421084965610138
8 0.9038964806924775
91 0.8845804438664042


In [122]:
# 1. Define an analytic device
dev_analytic = qml.device("default.qubit", wires=num_qubits, shots=None)

# 2. Define an identical circuit that returns the statevector
@qml.qnode(dev_analytic)
def state_circuit(params):
    qml.RY(params[0], wires=[num_qubits - 3])
    qml.RY(params[1], wires=[num_qubits - 1])
    qml.CRY(params[2], wires=[num_qubits - 1, num_qubits - 2])
    qml.RY(params[3], wires=[num_qubits - 2])
    qml.RY(params[4], wires=[num_qubits - 1])
    return qml.state()


In [123]:
lowest = 1e9
best_state = None
best_params = None

for i in range(1024):
    e = circuit(params)
    if e < lowest:
        lowest = e
        best_params = params.copy()
        best_state = state_circuit(params)
        print(f"New best energy: {lowest}")

New best energy: 0.9443810651215749
New best energy: 0.9075219581738296
New best energy: 0.8941355061520195
New best energy: 0.8902643443795499
New best energy: 0.8884516056388738
New best energy: 0.8863931826070803
New best energy: 0.8845804438664042


In [124]:
best_state

array([-0.80034271+0.j,  0.57023138+0.j, -0.13610105+0.j, -0.09730125+0.j,
        0.06370942+0.j, -0.04539195+0.j,  0.01083401+0.j,  0.00774544+0.j,
        0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
        0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j])